In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip install pygame

In [2]:
import pandas as pd
import streamlit as st
import re

In [3]:
job_settings = {
    "search_terms": "Data Scientist, AI Engineer, Machine Learning Engineer",
    "search_location": "United Kingdom",
    "switch_number": 8,
    "randomize": True,
    "sort_by": "Most recent",
    "date_posted": "Past week",
    "salary": 0,
    "experience_level": [],
    "job_type": [],
    "workplace_type": [],
    "preferred_companies": "",
    "location": "",
    "industry": "",
    "job_function": "",
    "job_titles": "",
    "benefits": "",
    "commitments": "",
    "under_10": False,
    "in_network": False,
    "fair_chance": False,
    "bad_words": "ACCENTURE, Google, Meta",
    "good_words": "",
    "job_desc_bad": "Security Clearance, DV Clearance, Driving Licence",
    "security_clearance": False,
    "has_masters": True,
    "current_experience": 3
}


In [4]:
export_path = "D:/Course/ChatBOT/logs/jobs_applied/linkedin_jobs.xlsx"
jobs_data = pd.read_excel(io=export_path, index_col=0)
jobs_data.head()

,Applied,Job Title,Company Name,Location,Date Posted,Applicants,Salary,Workplace Type,Job Type,Skills,Job Description,Recruiters,About Company,Application Type,Application Label,Application Link
Job ID,,,,,,,,,,,,,,,,
3668305504,Not Applied,IoT/Data Platform Engineer,Canonical,NaN,NaN,NaN,NaN,Remote,Full-time,"['Application Programming Interfaces (API)', '...",About the job\nCanonical is a leading provider...,[],"We deliver open source to the world faster, mo...",External,Apply to IoT/Data Platform Engineer on company...,https://job-boards.greenhouse.io/canonical/job...
4123134184,Not Applied,"Scientist, Machine Learning",Altos Labs,"Cambridge, England, United Kingdom",NaN,NaN,NaN,On-site,Full-time,[],About the job\nOur Mission\n\nOur mission is t...,[],Altos Labs is a biotechnology company focused ...,External,"Apply to Scientist, Machine Learning on compan...",https://job-boards.greenhouse.io/altoslabs/job...
4152986379,Not Applied,"Senior Machine Learning Engineer, Pricing",Zego,"London, England, United Kingdom",NaN,Over 100 applicants,NaN,Hybrid,Full-time,[],"About the job\nDescription\n\nAt Zego, we know...",[],Zego is a motor insurance provider with a visi...,Easy Apply,Easy Apply to Senior Machine Learning Engineer...,https://www.linkedin.com/jobs/search/?currentJ...
4164914883,Not Applied,"Research Engineer, ML, AI & Computer Vision",Meta,"London, England, United Kingdom",NaN,NaN,NaN,NaN,Full-time,[],About the job\nMeta Reality Labs Research (RL ...,[],Meta's mission is to build the future of human...,External,"Apply to Research Engineer, ML, AI &amp; Compu...",https://www.metacareers.com/jobs/6369754654907...
4164914883,Viewed,"Research Engineer, ML, AI & Computer Vision",Meta,"London, England, United Kingdom",NaN,NaN,NaN,NaN,Full-time,[],About the job\nMeta Reality Labs Research (RL ...,[],Meta's mission is to build the future of human...,External,"Apply to Research Engineer, ML, AI &amp; Compu...",https://www.metacareers.com/jobs/6369754654907...


In [5]:
def data_cleaning(jobs_data, job_settings, log_base="logs/job_application_logs/logs_text/", echo=False):
    # Remove duplicates
    before_dedup = len(jobs_data)
    jobs_data = jobs_data.drop_duplicates(subset=["Job ID", "Application Link"])
    after_dedup = len(jobs_data)
    print(f"🧹 Removed {before_dedup - after_dedup} duplicate rows based on Job ID and Application Link.")

    # Prepare filtered word lists
    bad_companies = [c.strip().lower() for c in job_settings.get("bad_words", "").split(",") if c.strip()]
    bad_phrases = [p.strip() for p in job_settings.get("job_desc_bad", "").split(",") if p.strip()]

    # Filter by company name
    if bad_companies:
        before_filter = len(jobs_data)
        jobs_data = jobs_data[~jobs_data["Company Name"].astype(str).str.lower().isin(bad_companies)]
        after_filter = len(jobs_data)
        print(f"🚫 Removed {before_filter - after_filter} jobs from bad companies.")

    # Filter by job description content
    if bad_phrases:
        pattern = '|'.join([re.escape(p) for p in bad_phrases])
        before_filter = len(jobs_data)
        jobs_data = jobs_data[~jobs_data["Job Description"].str.contains(pattern, case=False, na=False)]
        after_filter = len(jobs_data)
        print(f"📄 Removed {before_filter - after_filter} jobs with bad phrases in description.")

    return jobs_data

job_keywords = ["bad_words", "good_words","job_desc_bad","security_clearance","has_masters","current_experience"]

jobs_data = data_cleaning(jobs_data.reset_index(), job_settings)
jobs_data.head()

🧹 Removed 78 duplicate rows based on Job ID and Application Link.
🚫 Removed 4 jobs from bad companies.
📄 Removed 6 jobs with bad phrases in description.


,Job ID,Applied,Job Title,Company Name,Location,Date Posted,Applicants,Salary,Workplace Type,Job Type,Skills,Job Description,Recruiters,About Company,Application Type,Application Label,Application Link
0,3668305504,Not Applied,IoT/Data Platform Engineer,Canonical,NaN,NaN,NaN,NaN,Remote,Full-time,"['Application Programming Interfaces (API)', '...",About the job\nCanonical is a leading provider...,[],"We deliver open source to the world faster, mo...",External,Apply to IoT/Data Platform Engineer on company...,https://job-boards.greenhouse.io/canonical/job...
1,4123134184,Not Applied,"Scientist, Machine Learning",Altos Labs,"Cambridge, England, United Kingdom",NaN,NaN,NaN,On-site,Full-time,[],About the job\nOur Mission\n\nOur mission is t...,[],Altos Labs is a biotechnology company focused ...,External,"Apply to Scientist, Machine Learning on compan...",https://job-boards.greenhouse.io/altoslabs/job...
2,4152986379,Not Applied,"Senior Machine Learning Engineer, Pricing",Zego,"London, England, United Kingdom",NaN,Over 100 applicants,NaN,Hybrid,Full-time,[],"About the job\nDescription\n\nAt Zego, we know...",[],Zego is a motor insurance provider with a visi...,Easy Apply,Easy Apply to Senior Machine Learning Engineer...,https://www.linkedin.com/jobs/search/?currentJ...
6,4176152367,Viewed,Data Engineer,Duel Tech,"Bristol, England, United Kingdom",NaN,NaN,NaN,Hybrid,Full-time,[],About the job\nHybrid: Remote/Bristol\n\nSalar...,"[{'name': 'Ibrahim Thomas', 'title': '3rd', 'p...",The Duel Brand Advocacy Platform enables brand...,External,Apply to Data Engineer on company website,https://careers.duel.tech/jobs/5593144-data-en...
8,4190266260,Not Applied,Senior Computer Vision Engineer - Photo AI,Canva,"London, England, United Kingdom",NaN,NaN,NaN,Hybrid,Full-time,[],About the job\nJob Description\n\nJoin the tea...,"[{'name': 'David N.', 'title': '2nd', 'profile...",We're a global online visual communications pl...,External,Apply to Senior Computer Vision Engineer - Pho...,https://jobs.smartrecruiters.com/Canva/6000000...


In [6]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Map of standardized sections to heading variants
section_map = {
    "about_the_job": [
        "About the Job", "Job Summary", "Overview", "Who We Are", "About Us"
    ],
    "role_responsibilities": [
        "Role Responsibilities", "Responsibilities", "Key Responsibilities", "Duties", "What You'll Do"
    ],
    "role_requirements": [
        "Role Requirements", "Requirements", "Qualifications", "Must-Have", "Experience"
    ],
    "preferred_qualifications": [
        "Nice-to-Have", "Preferred Qualifications", "Highly Desirable", "Desirable Skills"
    ],
    "benefits": [
        "Benefits", "What We Offer", "Perks", "Compensation & Benefits"
    ]
}

# Setup Matcher
matcher = Matcher(nlp.vocab)
for section, headings in section_map.items():
    for heading in headings:
        pattern = [{"LOWER": token.lower()} for token in heading.split()]
        matcher.add(section, [pattern])

# Extract sections from a single job description
def extract_sections(text):
    if not isinstance(text, str) or not text.strip():
        return {}

    doc = nlp(text)
    matches = matcher(doc)
    matches = sorted(matches, key=lambda x: x[1])  # Sort by position

    sections = {}
    for i, (match_id, start, end) in enumerate(matches):
        label = nlp.vocab.strings[match_id]
        start_char = doc[start].idx
        end_char = doc[matches[i + 1][1]].idx if i + 1 < len(matches) else len(text)

        # Skip if the section is empty or too short (e.g. heading with no content)
        if end_char - start_char < 10:
            continue

        section_text = text[start_char:end_char].strip()

        if label in sections:
            sections[label] += "\n" + section_text
        else:
            sections[label] = section_text

    return sections


# Apply to each row of Job Description
parsed_sections = jobs_data["Job Description"].fillna("").apply(extract_sections)

# Flatten dictionary into columns
parsed_df_reset = pd.json_normalize(parsed_sections).reset_index(drop=True)
jobs_data_reset = jobs_data.reset_index(drop=True)

# Combine with original DataFrame
final_df = pd.concat([jobs_data_reset, parsed_df_reset], axis=1)

# Optional: View available columns
print(final_df.columns)

Index(['Job ID', 'Applied', 'Job Title', 'Company Name', 'Location',
       'Date Posted', 'Applicants', 'Salary', 'Workplace Type', 'Job Type',
       'Skills', 'Job Description', 'Recruiters', 'About Company',
       'Application Type', 'Application Label', 'Application Link',
       'about_the_job', 'benefits', 'role_requirements',
       'role_responsibilities', 'preferred_qualifications'],
      dtype='object')


In [7]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 5)
short_wait = WebDriverWait(driver, 1)

In [161]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

import time
import pyautogui
import pygame


# === Setup ===
profile = {
    "email": "ramu@gmail.com",
    "password": "SweepEnd4091##",
    "mobie": "+44 7459146448"
}


#------------------ Define dummy data for each field ----------------
my_information_data = {
    "source--source": {
        "type": "listbox",
        "value": "Direct Mail",
        "alternate_ids": [
            "sourceSection_source",
        ],
        "needs_done_button": True 
    },
    "candidateIsPreviousWorker": {
        "type": "radio",
        "value": "No",
        "alternate_ids": [
            "previousWorker_candidateIsPreviousWorker"
        ],
        "needs_done_button": False,
    },
    "country--country": {
        "type": "listbox",
        "value": "United Kingdom",
        "alternate_ids": [
            "countryDropdown",
        ],
        "needs_done_button": True,
    },
    "name--legalName--title": {
        "type": "listbox",
        "value": "Mr",
        "alternate_ids": [
            "legalNameSection_title",
        ],
        "needs_done_button": False,
    },
    "name--legalName--firstName": {
        "type": "text",
        "value": "Don",
        "alternate_ids": [
            "legalNameSection_firstName",
        ],
        "needs_done_button": False,
    },
    "name--legalName--lastName": {
        "type": "text",
        "value": "Aaja",
        "alternate_ids": [
            "legalNameSection_lastName",
        ],
        "needs_done_button": False,
    },
    "name--preferredCheck": {
        "type": "checkbox",
        "value": True,
        "alternate_ids": [
            "preferredNameCheckbox",
        ],
        "needs_done_button": False,
    },
    "name--preferredName--title": {
        "type": "listbox",
        "value": "Dr",
        "alternate_ids": [
            "preferredNameSection_title",
        ],
        "needs_done_button": False,
    },
    "name--preferredName--firstName": {
        "type": "text",
        "value": "Ramu",
        "alternate_ids": [
            "preferredNameSection_firstName",
        ],
        "needs_done_button": False,
    },
    "name--preferredName--lastName": {
        "type": "text",
        "value": "Kaka",
        "alternate_ids": [
            "preferredNameSection_lastName",
        ],
        "needs_done_button": False,
    },
    "address--addressLine1": {
        "type": "text",
        "value": "1423 Main St",
        "alternate_ids": [
            "addressSection_addressLine1",
        ],
        "needs_done_button": False,
    },
    "address--addressLine2": {
        "type": "text",
        "value": "Sui5te 456",
        "alternate_ids": [
            "addressSection_addressLine2",
        ],
        "needs_done_button": False,
    },
    "address--addressLine3": {
        "type": "text",
        "value": "Bu8ilding A",
        "alternate_ids": [
            "addressSection_addressLine3",
        ],
        "needs_done_button": False,
    },
    "address--city": {
        "type": "text",
        "value": "Notting8ham",
        "alternate_ids": [
            "addressSection_city",
        ],
        "needs_done_button": False,
    },
    "address--countryRegion": {
        "type": "listbox",
        "value": "Nottinghamshire",
        "alternate_ids": [
            "addressSection_countryRegion",
        ],
        "needs_done_button": False,
    },
    "address--postalCode": {
        "type": "text",
        "value": "NG1 2DB",
        "alternate_ids": [
            "addressSection_postalCode",
        ],
        "needs_done_button": False,
    },
    "phoneNumber--phoneType": {
        "type": "listbox",
        "value": "Mobile",
        "alternate_ids": [
            "phone-device-type",
        ],
        "needs_done_button": False,
    },
    "phoneNumber--countryPhoneCode": {
        "type": "text",
        "value": "United Kingdom (+44)",
        "alternate_ids": [
            "country-phone-code",
        ],
        "needs_done_button": True,
    },
    "phoneNumber--phoneNumber": {
        "type": "text",
        "value": "7469137449",
        "alternate_ids": [
            "phone-number",
        ],
        "needs_done_button": False,
    }
}


my_experience_data = {
    "workExperience": [
        {
            "jobTitle": "Software Engineer",
            "companyName": "ABC Corp",
            "location": "Bangalore",
            "currentlyWorkHere": True,
            "startDate": "28/01/2020",
            "endDate": "31/12/2022",
            "roleDescription": "Developed web applications and maintained software systems."
        },
        {
            "jobTitle": "Senior Developer",
            "companyName": "XYZ Ltd",
            "location": "London",
            "currentlyWorkHere": False,
            "startDate": "28/02/2017",
            "endDate": "31/12/2019",
            "roleDescription": "Led a team of developers and implemented enterprise solutions."
        },
        {
            "jobTitle": "Full Stack Developer",
            "companyName": "Tech Solutions",
            "location": "Remote",
            "currentlyWorkHere": False,
            "startDate": "01/03/2015",
            "endDate": "01/01/2017",
            "roleDescription": "Built web and mobile applications for clients worldwide."
        },
        {
            "jobTitle": "Software Intern",
            "companyName": "Startup Inc.",
            "location": "Berlin",
            "currentlyWorkHere": False,
            "startDate": "01/06/2014",
            "endDate": "01/02/2015",
            "roleDescription": "Assisted with developing internal tools and automations."
        },
        {
            "jobTitle": "Junior Developer",
            "companyName": "Innovatech",
            "location": "New York",
            "currentlyWorkHere": False,
            "startDate": "01/01/2013",
            "endDate": "01/05/2014",
            "roleDescription": "Worked on backend systems and contributed to product launches."
        }
    ],
    "education": [
        {
            "schoolName": "University of Nottingham",
            "degree": "Bachelor of Engineering",
            "fieldOfStudy": "Computer Science",
            "gradeAverage": "Merit",
            "startDate": "01/09/2016",
            "endDate": "01/06/2020"
        },
        {
            "schoolName": "University of Brighsa",
            "degree": "Masters",
            "fieldOfStudy": "Mathematics",
            "gradeAverage": "Merit",
            "startDate": "01/09/2016",
            "endDate": "01/06/2020"
        }
    ],
    "certifications": [
        {
            "certification": "AWS Certified Solutions Architect",
            "certificationNumber": "AWS-12345",
            "issuedDate": "15/06/2022",
            "expirationDate": "15/06/2025",
            "attachments": r"C:\Users\ajayc\Downloads\Ajay Hiremath  Resume.pdf"
        },
        {
            "certification": "Google Cloud Certified",
            "certificationNumber": "GCP-67890",
            "issuedDate": "20/03/2021",
            "expirationDate": "20/03/2024",
            "attachments": r"C:\Users\ajayc\Downloads\Ajay Hiremath Other Work Experience.docx"
        }
    ],    
    "languages": [
        {
        "language": "English",
        "native": True,
        "8c5fc5940f0b01f097f087dbc3009062": "Fluent"
        },
        {
        "language": "French",
        "native": False,
        "8c5fc5940f0b01f097f087dbc3009062": "Intermediate"
        },
    ],
    "skills-section": [
        {
        "skills": ["Python", "SQL", "R", "Pandas", "NumPy", "Geopandas", "Matplotlib", "Seaborn", "Plotly", 
                   "Power BI", "R Shiny", "OpenCV", "Power Automate", "TensorFlow", "PyTorch", 
                   "Scikit-Learn", "SHAP", "XGBoost", "AdaBoost", "Lambda", "SageMaker", 
                   "Glue", "EC2", "S3", "API Gateway", "Redshift", "Spark", "Hadoop", 
                   "Hive", "Kafka", "Flume", "HBase", "Distributions", "Multivariate Analysis", 
                   "GLMs", "Hypothesis Testing", "A/B Testing", "Chi-Square", "ANOVA", 
                   "T-Tests", "ARIMA", "SARIMA", "SARIMAX", "VARMAX", "Prophet", 
                   "Seasonal Decomposition", "Rolling Statistics", "SpaCy", "NLTK", 
                   "GANs", "LLMs", "Prompt Engineering", "RAG", "Multimodal AI", "AI Agents", 
                   "LangChain", "LCEL", "Hugging Face", "FAISS", "Chroma", "Pinecone", "Astra DB", 
                   "Git", "Flask", "Databricks", "Streamlit", "Predictive Modelling", "Computer Vision", 
                   "NLP", "Fraud Detection", "Healthcare Analytics", "Geospatial Modelling", 
                   "Retail & E-commerce", "Insurance", "Customer Analytics", "Real-Time Analytics", 
                   "Distributed Computing"]
        },
    ],
    "resume": [
        {
        "attachments" : r"C:\Users\ajayc\Downloads\Ajay Hiremath  Resume.pdf",
        },
    ],
}

workexperience_field_mapping = {"jobTitle": "text",
                               "companyName": "text",
                               "location": "text",
                               "currentlyWorkHere": "checkbox",
                               "startDate-dateSectionDay": "date",
                               "startDate-dateSectionMonth": "date",
                               "startDate-dateSectionYear": "date",
                               "endDate-dateSectionDay": "date",
                               "endDate-dateSectionMonth": "date",
                               "endDate-dateSectionYear": "date",
                               "roleDescription": "text"
                               }

certification_field_mapping = {"certification": "listbox",
                              "certificationNumber": "text",
                              "issuedDate-dateSectionDay": "date",
                              "issuedDate-dateSectionMonth": "date",
                              "issuedDate-dateSectionYear": "date",
                              "expirationDate-dateSectionDay": "date",
                              "expirationDate-dateSectionMonth": "date",
                              "expirationDate-dateSectionYear": "date",
                              "attachments": "file"
                              }

education_field_mapping = {"schoolName": "text",
                           "degree": "listbox",
                           "fieldOfStudy": "multiselect",
                           "gradeAverage": "text",
                           "startDate-firstYearAttended-dateSectionDay": "date",
                           "startDate-firstYearAttended-dateSectionMonth": "date",
                           "startDate-firstYearAttended-dateSectionYear": "date",
                           "endDate-lastYearAttended-dateSectionDay": "date",
                           "endDate-lastYearAttended-dateSectionMonth": "date",
                           "endDate-lastYearAttended-dateSectionYear": "date"
                           }
language_field_mapping = {
    "language": "listbox",
    "native": "checkbox",
    "8c5fc5940f0b01f097f087dbc3009062": "listbox"
}

skill_field_mapping = {
    "skills": "multiselect"
}

resume_field_mapping = {
    "attachments": "file"
}

In [9]:
# #-------------------{Function to open URL in Workday and clear local/session storage}---------------------------
# def send_workday_url(driver, url):
#     #-------------------{Navigate to URL}---------------------------
#     driver.get(url)
#     time.sleep(2)
#     #-------------------{Clear cookies and local storage for a clean session}---------------------------
#     driver.delete_all_cookies()
#     driver.execute_script("window.localStorage.clear();")
#     driver.execute_script("window.sessionStorage.clear();")
#     driver.refresh()
#     time.sleep(1)

# #-------------------{Function to detect missing job page error}---------------------------
# def is_job_page_missing(driver, short_wait, log_base="logs/job_application_logs/logs_text/", echo=False):
#     try:
#         #-------------------{Wait for error message element}---------------------------
#         short_wait.until(lambda d: d.find_elements(By.XPATH, "//span[@data-automation-id='errorMessage']"))
#         elements = driver.find_elements(By.XPATH, "//span[@data-automation-id='errorMessage']")
#         for el in elements:
#             #-------------------{Check if error message indicates missing job}---------------------------
#             if "doesn't exist" in el.text or "not available" in el.text.lower():
#                 print(f"❌ Job error found: {el.text.strip()} — skipping.")
#                 return True
#     except TimeoutException:
#         pass
#     return False

# #-------------------{Function to click apply button}---------------------------
# def click_apply_buttons(driver, wait, xpath, log_base="logs/job_application_logs/logs_text/", echo=False):
#     try:
#         print(f"Trying to click: {xpath}")
#         button = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
#         driver.execute_script("arguments[0].scrollIntoView(true);", button)
#         time.sleep(1)
#         driver.execute_script("arguments[0].click();", button)
#         print("✔ Click successful.")
#         return True
#     except (TimeoutException, NoSuchElementException) as e:
#         print(f"⚠️ Click failed for xpath: {xpath} — {e}")
#         return False

# #-------------------{Function to tick consent checkbox if present}---------------------------
# def tick_checkbox_if_present(driver, log_base="logs/job_application_logs/logs_text/", echo=False):
#     try:
#         checkbox = driver.find_element(By.CSS_SELECTOR, "input[data-automation-id='createAccountCheckbox']")
#         if not checkbox.is_selected():
#             driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
#             time.sleep(0.5)
#             driver.execute_script("arguments[0].click();", checkbox)
#             print("✔ Ticked consent checkbox.")
#             return True
#     except NoSuchElementException:
#         print("ℹ️ Consent checkbox not found — skipping.")
#     return False

# #-------------------{Function to check if page is Sign-In}---------------------------
# def is_signin_page():
#     return bool(driver.find_elements(By.XPATH, "//h2[contains(text(), 'Sign In')]"))

# #-------------------{Function to check if page is Create Account}---------------------------
# def is_create_account_page():
#     return bool(driver.find_elements(By.XPATH, "//h2[contains(text(), 'Create Account')]"))

# #-------------------{Function to click Sign-In switch button}---------------------------
# def try_click_signin_switch(wait, log_base="logs/job_application_logs/logs_text/", echo=False):
#     try:
#         switch_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-automation-id='signInLink']")))
#         driver.execute_script("arguments[0].click();", switch_button)
#         print("🔁 Switched to Sign-In form.")
#         time.sleep(2)
#     except TimeoutException:
#         print("⚠️ Switch to Sign-In button not found.")

# #-------------------{Function to click Create Account switch button}---------------------------
# def try_click_create_account_switch(wait, log_base="logs/job_application_logs/logs_text/", echo=False):
#     try:
#         switch_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-automation-id='createAccountLink']")))
#         driver.execute_script("arguments[0].click();", switch_button)
#         print("🔁 Switched to Create Account form.")
#         time.sleep(2)
#     except TimeoutException:
#         print("⚠️ Switch to Create Account button not found.")

# #-------------------{Function to fill email and password fields}---------------------------
# def send_email_password(driver, wait, profile, verify_password=False, log_base="logs/job_application_logs/logs_text/", echo=False):
#     #-------------------{Fill email field}---------------------------
#     email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='email'], input[data-automation-id='email']")))
#     email_input.send_keys(Keys.CONTROL + "a", Keys.DELETE)
#     email_input.send_keys(profile['email'])
#     print("📧 Filled email.")

#     #-------------------{Fill password field}---------------------------
#     password_input = driver.find_element(By.CSS_SELECTOR, "input[type='password'][data-automation-id='password'], input[type='password'][name='password']")
#     password_input.send_keys(Keys.CONTROL + "a", Keys.DELETE)
#     password_input.send_keys(profile['password'])
#     print("🔒 Filled password.")

#     #-------------------{Fill confirm password if required}---------------------------
#     if verify_password:
#         confirm_password_input = driver.find_element(By.CSS_SELECTOR, "input[type='password'][data-automation-id='verifyPassword'], input[type='password'][name='verifyPassword']")
#         confirm_password_input.send_keys(Keys.CONTROL + "a", Keys.DELETE)
#         confirm_password_input.send_keys(profile['password'])
#         print("🔒 Confirmed password.")

#     #-------------------{Tick consent checkbox if present}---------------------------
#     tick_checkbox_if_present(driver)

# #-------------------{Function to play beep sound as an alert}---------------------------
# def alert_user_beep(sound_file='D:/Course/ChatBOT/components/linkedin_automation/jobs_applier_selanium/notification/alert.mp3', repeat=3):
#     try:
#         pygame.mixer.init()
#         for _ in range(repeat):
#             pygame.mixer.music.load(sound_file)
#             pygame.mixer.music.play()
#             while pygame.mixer.music.get_busy():
#                 pygame.time.Clock().tick(10)
#     except Exception:
#         print(f"⚠️ Failed to play sound")

# #-------------------{Function to handle manual authentication flow}---------------------------
# def handle_manual_authentication():
#     alert_user_beep()
#     print("⌛ Waiting for user manual authentication confirmation...")
#     start_time = time.time()
#     while time.time() - start_time < 300:
#         response = pyautogui.confirm(
#             text='Have you completed manual authentication?',
#             title='Manual Auth Required',
#             buttons=['Yes', 'No']
#         )
#         if response == 'Yes':
#             print("✔ User confirmed authentication.")
#             return True
#         elif response == 'No':
#             print("⏭️ User declined. Skipping job.")
#             return False
#         else:
#             time.sleep(5)
#     print("⏳ Timeout: No manual authentication confirmed.")
#     return False


# #-------------------{Function to handle sign-up or sign-in flow}---------------------------
# def signup_or_signin(driver, wait, profile, timeout_seconds=60, log_base="logs/job_application_logs/logs_text/", echo=False):
#     #-------------------{Start signup/signin attempt with a timer}---------------------------
#     print("Attempting to sign up or sign in...")
#     start_time = time.time()
#     try:
#         while time.time() - start_time < timeout_seconds:
#             driver.switch_to.default_content()

#             #-------------------{If on Create Account page}---------------------------
#             if is_create_account_page():
#                 #-------------------{Fill email and password fields with verification}---------------------------
#                 send_email_password(driver, wait, profile, verify_password=True)
#                 try:
#                     create_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Create Account']")))
#                     driver.execute_script("arguments[0].click();", create_btn)
#                     print("📝 Attempted account creation.")
#                 except Exception:
#                     print(f"⚠️ Could not click 'Create Account'")
#                 time.sleep(2)

#                 #-------------------{Check if redirected to Sign In page after creation attempt}---------------------------
#                 if is_signin_page():
#                     if handle_manual_authentication():
#                         send_email_password(driver, wait, profile, verify_password=False)
#                         try:
#                             signin_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Sign In']")))
#                             driver.execute_script("arguments[0].click();", signin_btn)
#                             print("🔐 Attempted sign in.")
#                         except Exception:
#                             print(f"⚠️ Could not click 'Sign In'")
#                         time.sleep(2)
#                     else:
#                         break

#             #-------------------{If on Sign In page}---------------------------
#             elif is_signin_page():
#                 send_email_password(driver, wait, profile, verify_password=False)
#                 try:
#                     signin_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Sign In']")))
#                     driver.execute_script("arguments[0].click();", signin_btn)
#                     print("🔐 Attempted sign in.")
#                 except Exception:
#                     print(f"⚠️ Could not click 'Sign In'")
#                 time.sleep(2)

#             #-------------------{Check if navigated to My Information page or Application Flow}---------------------------
#             if (
#                 driver.find_elements(By.XPATH, "//h1[contains(text(), 'Something went wrong')]") or
#                 driver.find_elements(By.XPATH, "//h2[contains(text(), 'My Information')]") or
#                 driver.find_elements(By.CSS_SELECTOR, "button[data-automation-id='pageFooterNextButton']")
#             ):
#                 print("✔ Success: Detected entry into application flow or profile page.")
#                 return True
#             else:
#                 #-------------------{If stuck, try toggling between Create Account and Sign In forms}---------------------------
#                 if is_create_account_page():
#                     try_click_signin_switch(wait)
#                 elif is_signin_page():
#                     try_click_create_account_switch(wait)

#             time.sleep(1)

#         #-------------------{Timeout reached if still on Sign In or Create Account page}---------------------------
#         print("⏳ Timeout: did not reach 'My Information' page.")
#         return False

#     except Exception:
#         print(f"❌ Error during signup/signin")
#         return False


# def my_information_page(driver, wait=WebDriverWait(driver, 2), form_data=form_data):
#     #-------------------{Refresh the page to ensure the latest DOM}---------------------------
#     driver.refresh()
#     time.sleep(5)

#     #-------------------{Build XPath for the 'My Information' section header}---------------------------
#     xpath_base = f"//div[@data-automation-id='applyFlowPage']//h2[contains(text(), 'My Information')]"

#     #-------------------{Try clicking 'Apply' buttons to reach the information page}---------------------------
#     if not click_apply_buttons(driver, wait, xpath_base):
#         print("Unable to find my information. Continuing with the next job")
#         return None

#     #-------------------{Initialize a dictionary to store fields that failed processing}---------------------------
#     failed_fields = {}

#     #-------------------{Helper function to locate an element by trying multiple IDs}---------------------------
#     def interact_with(driver, wait, condition, by, id_list):
#         try:
#             #-------------------{Build a list of conditions to wait for any matching ID}---------------------------
#             all_conditions = []
#             for id_option in id_list:
#                 all_conditions.append(condition((by, id_option)))
#                 all_conditions.append(condition((By.XPATH, f"//*[@data-automation-id='{id_option}']")))
#             wait.until(EC.any_of(*all_conditions))

#             #-------------------{Iterate through IDs and attempt to find the element}---------------------------
#             for id_option in id_list:
#                 try:
#                     element = driver.find_element(by, id_option)
#                 except:
#                     try:
#                         element = driver.find_element(By.XPATH, f"//*[@data-automation-id='{id_option}']")
#                     except:
#                         continue
#                 driver.execute_script("arguments[0].scrollIntoView(true);", element)
#                 time.sleep(0.2)

#                 #-------------------{Try to find an input, button, or label inside the element}---------------------------
#                 try:
#                     return element.find_element(By.XPATH, ".//button|.//input|.//label"), id_option
#                 except:
#                     return element, id_option

#         except Exception:
#             raise Exception(f"Element not found for IDs: {id_list}")

#     #-------------------{Handler for text fields}---------------------------
#     def handle_text(driver, wait, all_ids, value, retry):
#         element, used_id = interact_with(driver, wait, EC.presence_of_element_located, By.ID, all_ids)
#         try:
#             input_elem = element.find_element(By.XPATH, ".//input")
#         except:
#             input_elem = element
#         input_elem.send_keys(Keys.CONTROL + 'a')
#         input_elem.send_keys(Keys.DELETE)
#         input_elem.send_keys(value)
#         input_elem.send_keys(Keys.TAB)
#         print(f"{'🔁 Retried' if retry else '✔'} Filled text '{used_id}' with '{value}'")
#         return True

#     #-------------------{Handler for listbox (dropdown) fields}---------------------------
#     def handle_listbox(driver, wait, all_ids, value, retry):
#         try:
#             element, used_id = interact_with(driver, wait, EC.element_to_be_clickable, By.ID, all_ids)
#         except:
#             element, used_id = interact_with(driver, wait, EC.element_to_be_clickable, By.XPATH,
#                                              [f"//button[@id='{id_option}']" for id_option in all_ids])

#         #-------------------{Attempt to find an input inside the dropdown}---------------------------
#         try:
#             input_elem = element.find_element(By.XPATH, ".//input")
#             input_elem.clear()
#         except:
#             pass

#         #-------------------{Click the button to open the dropdown}---------------------------
#         driver.execute_script("arguments[0].click();", element)
#         time.sleep(0.3)

#         #-------------------{Send each character of the value (simulate typing)}---------------------------
#         for char in value:
#             element.send_keys(char)

#         #-------------------{Press ENTER to select the value}---------------------------
#         element.send_keys(Keys.ENTER)

#         print(f"{'🔁 Retried' if retry else '✔'} Selected '{value}' in listbox '{used_id}'")

#         #-------------------{Click the 'Done' button if it exists}---------------------------
#         try:
#             done_button = driver.find_element(By.XPATH, "//button[.//span[text()='Done']]")
#             driver.execute_script("arguments[0].scrollIntoView(true);", done_button)
#             driver.execute_script("arguments[0].click();", done_button)
#             time.sleep(0.5)
#         except Exception:
#             pass

#         return True

#     #-------------------{Handler for radio button fields}---------------------------
#     def handle_radio(driver, wait, field_id, value, retry):
#         wait.until(EC.presence_of_all_elements_located((By.NAME, field_id)))
#         radios = driver.find_elements(By.NAME, field_id)
#         for radio in radios:
#             radio_id = radio.get_attribute("id")
#             if not radio_id:
#                 continue
#             try:
#                 label_elem = driver.find_element(By.XPATH, f"//label[@for='{radio_id}']")
#                 label_text = label_elem.text.strip().lower()
#                 if value.lower() == label_text:
#                     is_checked = radio.get_attribute("aria-checked") == "true" or radio.is_selected()
#                     if is_checked:
#                         print(f"{'🔁 Retried' if retry else '✔'} Radio '{label_text}' already selected for '{field_id}'")
#                         return True
#                     driver.execute_script("arguments[0].scrollIntoView(true);", radio)
#                     driver.execute_script("arguments[0].click();", radio)
#                     time.sleep(1)
#                     print(f"{'🔁 Retried' if retry else '✔'} Selected radio '{label_text}' for '{field_id}'")
#                     return True
#             except Exception:
#                 print(f"⚠️ Could not find label for radio id '{radio_id}'")
#         print(f"{'⚠️ Retried' if retry else '❌'} No matching radio label for value '{value}' in '{field_id}'")
#         return False

#     #-------------------{Handler for checkbox fields}---------------------------
#     def handle_checkbox(driver, wait, all_ids, value, retry):
#         element, used_id = interact_with(driver, wait, EC.presence_of_element_located, By.ID, all_ids)
#         is_checked = element.is_selected()
#         if value != is_checked:
#             driver.execute_script("arguments[0].click();", element)
#         print(f"{'🔁 Retried' if retry else '✔'} Checkbox '{used_id}' set to '{value}'")
#         return True

#     #-------------------{Dispatcher to call the correct handler based on field type}---------------------------
#     def try_field_all_ids(field_id, config, retry=False):
#         field_type = config.get("type")
#         value = config.get("value")
#         all_ids = [field_id] + config.get("alternate_ids", [])

#         try:
#             if field_type == "text":
#                 return handle_text(driver, wait, all_ids, value, retry)
#             elif field_type == "listbox":
#                 return handle_listbox(driver, wait, all_ids, value, retry)
#             elif field_type == "radio":
#                 return handle_radio(driver, wait, field_id, value, retry)
#             elif field_type == "checkbox":
#                 return handle_checkbox(driver, wait, all_ids, value, retry)
#         except Exception:
#             print(f"{'⚠️ Retried' if retry else '❌'} Failed to handle '{field_id}' ({field_type})")
#         return False

#     #-------------------{First pass: try to fill each field}---------------------------
#     for field_id, config in form_data.items():
#         if not try_field_all_ids(field_id, config):
#             failed_fields[field_id] = config

#     #-------------------{Retry pass: retry any fields that failed on the first attempt}---------------------------
#     if failed_fields:
#         print("\n🔁 Retrying failed fields...\n")
#         for field_id, config in failed_fields.items():
#             if not try_field_all_ids(field_id, config, retry=True):
#                 print(f"⚠️ Field '{field_id}' still failed after retry.")

#     #-------------------{Return the dictionary of fields that failed even after retry}---------------------------
#     return failed_fields

# #--------------------my_experience_form---------------------------------
# def my_experience_form(driver, form_data_page_2):
#     #--------------------Initialize Waits---------------------------------
#     wait = WebDriverWait(driver, 5)
#     short_wait = WebDriverWait(driver, 0.5)
#     time.sleep(2)
#     print("🌟 Starting Page 2 filling process...")

#     #--------------------Helper to delete files---------------------------------
#     def delete_existing_files(driver):
#         delete_buttons = driver.find_elements(By.XPATH, "//button[@type='button' and @data-automation-id='delete-file']")
#         for btn in delete_buttons:
#             try:
#                 driver.execute_script("arguments[0].scrollIntoView(true);", btn)
#                 time.sleep(0.2)
#                 driver.execute_script("arguments[0].click();", btn)
#                 print("✔ Deleted existing file.")
#                 time.sleep(1)  # allow UI to update
#             except Exception as e:
#                 print(f"⚠️ Could not delete file: {e}")

#     #--------------------Helper to delete selected pills in multiselect---------------------------------
#     def delete_selected_pills(driver):
#         delete_icons = driver.find_elements(By.XPATH, "//div[@data-automation-id='DELETE_charm']")
#         for icon in delete_icons:
#             try:
#                 driver.execute_script("arguments[0].scrollIntoView(true);", icon)
#                 time.sleep(0.2)
#                 driver.execute_script("arguments[0].click();", icon)
#                 print("✔ Cleared a selected value.")
#             except Exception as e:
#                 print(f"⚠️ Could not click delete icon: {e}")

#     #--------------------interact_with---------------------------------
#     def interact_with(driver, wait, by, elem_name, value, elem_type="text", date_field_crt=None):
#         try:
#             #--------------------Locate element---------------------------------
#             if elem_type == "text":
#                 element = wait.until(EC.visibility_of_element_located((by, elem_name)))
#             elif elem_type in ["checkbox", "listbox", "multiselect"]:
#                 element = wait.until(EC.presence_of_element_located((by, elem_name)))
#             elif elem_type == "file":
#                 print(elem_name)
#                 element = wait.until(EC.presence_of_element_located((by, elem_name)))
#             elif elem_type == "date":
#                 x_path, elem_name = elem_name.split("-x-")
#                 if date_field_crt:
#                     elem_name = date_field_crt + "-" + elem_name.split("-")[-1]
#                 display_xpath = f"{x_path}//div[contains(@id,'{elem_name}-display')]"
#                 input_xpath = f"{x_path}//input[contains(@id,'{elem_name}-input')]"
#                 try:
#                     display_element = wait.until(EC.visibility_of_element_located((By.XPATH, display_xpath)))
#                     driver.execute_script("arguments[0].scrollIntoView(true);", display_element)
#                     driver.execute_script("arguments[0].click();", display_element)
#                     time.sleep(0.3)
#                 except:
#                     print(f"⚠️ Display element for {elem_name} not found or not clickable. Proceeding to input.")
#                 element = wait.until(EC.visibility_of_element_located((By.XPATH, input_xpath)))
#             else:
#                 print(f"⚠️ Unknown element type '{elem_type}' for {elem_name}. Skipped.")
#                 return None

#             driver.execute_script("arguments[0].scrollIntoView(true);", element)
#             time.sleep(0.3)

#             #--------------------Handle element types---------------------------------
#             if elem_type == "checkbox":
#                 return element
#             elif elem_type == "file":
#                 delete_existing_files(driver)
#                 try:
#                     driver.execute_script("""
#                                           arguments[0].style.display = 'block';
#                                           arguments[0].style.visibility = 'visible';
#                                           arguments[0].style.opacity = '1';
#                                           arguments[0].style.position = 'static';
#                                           """, element)
#                     element.send_keys(value)
#                     time.sleep(5)
#                     print(f"✔ Uploaded file: {value}")
#                 except:
#                     print(f"⚠️ Could not upload file")
#             elif elem_type == "multiselect":
#                 delete_selected_pills(driver)
#                 driver.execute_script("arguments[0].click();", element)
#                 element.send_keys(Keys.CONTROL, 'a')
#                 element.send_keys(Keys.DELETE)
#                 time.sleep(0.2)
#                 element.send_keys(value)
#                 time.sleep(1)
#                 element.send_keys(Keys.ENTER)
#                 try:
#                     dropdown_option_xpath = f"//div[@data-automation-id='promptOption' and @data-automation-label='{value}']"
#                     dropdown_option = wait.until(EC.visibility_of_element_located((By.XPATH, dropdown_option_xpath)))
#                     driver.execute_script("arguments[0].scrollIntoView(true);", dropdown_option)
#                     driver.execute_script("arguments[0].click();", dropdown_option)
#                     print(f"✔ Selected multiselect option: {value}")
#                 except Exception as e:
#                     print(f"⚠️ Could not select dropdown option: {e}")
#                 element.send_keys(Keys.TAB)
#             elif elem_type in ["text", "listbox", "date"]:
#                 if elem_type == "date":
#                     element.clear()
#                     time.sleep(0.1)
#                 else:
#                     driver.execute_script("arguments[0].click();", element)
#                     element.send_keys(Keys.CONTROL, 'a')
#                     time.sleep(0.1)
#                     element.send_keys(Keys.DELETE)
#                     time.sleep(0.1)
#                     if elem_type != "listbox":
#                         element.clear()
#                     time.sleep(0.2)
#                 if elem_type == "date":
#                     for char in value:
#                         element.send_keys(char)
#                 else:
#                     element.send_keys(value)
#                 time.sleep(1)
#                 if elem_type == "listbox":
#                     element.send_keys(Keys.ENTER)
#                 if elem_type != "date":
#                     element.send_keys(Keys.TAB)
#                 print(f"✔ Filled {elem_name} with '{value}'")
#         except TimeoutException:
#             print(f"⚠️ Could not fill {elem_name}")
#             return None

#     #--------------------fill_experience_or_education_sections---------------------------------
#     def fill_experience_or_education_sections(driver, wait, data_list, section_name, field_mapping):
#         try:
#             add_button_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}section']//button[@data-automation-id='add-button' and (text()='Add')]"
#             click_apply_buttons(driver, wait, add_button_xpath)
#             print(f"✔ Clicked 'Add' for {section_name}.")
#         except Exception:
#             print(f"⚠️ Failed to click 'Add' button for {section_name}.")

#         for index, entry in enumerate(data_list, start=1):
#             try:
#                 section_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}{index}-panel']"
#                 time.sleep(2)
#                 try:
#                     h4_elements = driver.find_elements(By.XPATH, f"//h4[@id='{section_name}{index}-panel']")
#                     if h4_elements and h4_elements[0].is_displayed():
#                         print(f"✔ {section_name}{index} section already exists. Skipping adding.")
#                     else:
#                         add_another_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}section']//button[contains(@data-automation-id, 'add-button') and (text()='Add Another')]"
#                         if click_apply_buttons(driver, wait, add_another_xpath):
#                             print(f"✔ Clicked 'Add Another' for {section_name}{index}.")
#                             time.sleep(2)
#                 except Exception as e:
#                     print(f"⚠️ Error clicking 'Add Another' for {section_name}{index}: {e}")

#                 for field_key, field_type in field_mapping.items():
#                     try:
#                         if field_type == "date":
#                             date_field_name = field_key.split('-')[0]
#                             date_value = entry.get(date_field_name, "")
#                             if not date_value:
#                                 continue
#                             day, month, year = date_value.split("/")
#                             if "dateSectionDay" in field_key:
#                                 value = day
#                             elif "dateSectionMonth" in field_key:
#                                 value = month
#                             elif "dateSectionYear" in field_key:
#                                 value = year
#                             else:
#                                 continue
#                             element_xpath = f"{section_xpath}-x-{field_key}"
#                             date_field_crt = None
#                             if len(field_key.split('-')) == 3:
#                                 date_field_crt = field_key.split('-')[1]
#                             interact_with(driver, short_wait, By.XPATH, element_xpath, value, "date", date_field_crt)
#                         elif field_type == "file":
#                             element_xpath = f"{section_xpath}//input[@data-automation-id='file-upload-input-ref' and @type='file']"
#                             interact_with(driver, wait, By.XPATH, element_xpath, entry[field_key], "file")
#                         elif field_type == "listbox":
#                             element_xpath = f"{section_xpath}//button[@type='button' and contains(@id,'--{field_key}') and contains(@name,'{field_key}')]"
#                             interact_with(driver, wait, By.XPATH, element_xpath, entry[field_key], "listbox")
#                         elif field_type == "multiselect":
#                             element_xpath = f"{section_xpath}//input[contains(@id,'--{field_key}') and @data-uxi-widget-type='selectinput']"
#                             interact_with(driver, wait, By.XPATH, element_xpath, entry[field_key], "multiselect")
#                         elif field_type == "checkbox":
#                             element_xpath = f"{section_xpath}//input[@type='checkbox' and contains(@id,'--{field_key}') and contains(@name,'{field_key}')]"
#                             checkbox = interact_with(driver, short_wait, By.XPATH, element_xpath, entry[field_key], "checkbox")
#                             if entry[field_key] != (checkbox.get_attribute("aria-checked") == "true"):
#                                 driver.execute_script("arguments[0].click();", checkbox)
#                                 print(f"✔ Set '{field_key}' checkbox for {section_name}{index}.")
#                         else:
#                             if field_key == "roleDescription":
#                                 element_xpath = f"{section_xpath}//textarea[contains(@id,'--{field_key}')]"
#                             else:
#                                 element_xpath = f"{section_xpath}//input[contains(@id,'--{field_key}') and contains(@name,'{field_key}')]"
#                             interact_with(driver, wait, By.XPATH, element_xpath, entry[field_key], field_type)
#                     except Exception as e:
#                         print(f"⚠️ Could not fill field {field_key} for {section_name}{index}: {e}")
#                 print(f"✔ Finished {section_name}{index}.")
#             except Exception as e:
#                 print(f"⚠️ Error filling {section_name}{index}: {e}")

#     #--------------------Fill Sections---------------------------------
#     fill_experience_or_education_sections(driver, wait,
#                                           data_list=form_data_page_2.get("workExperience", []),
#                                           section_name="Work-Experience-",
#                                           field_mapping={
#                                               "jobTitle": "text",
#                                               "companyName": "text",
#                                               "location": "text",
#                                               "currentlyWorkHere": "checkbox",
#                                               "startDate-dateSectionDay": "date",
#                                               "startDate-dateSectionMonth": "date",
#                                               "startDate-dateSectionYear": "date",
#                                               "endDate-dateSectionDay": "date",
#                                               "endDate-dateSectionMonth": "date",
#                                               "endDate-dateSectionYear": "date",
#                                               "roleDescription": "text"
#                                           })

#     fill_experience_or_education_sections(driver, wait,
#                                           data_list=form_data_page_2.get("education", []),
#                                           section_name="Education-",
#                                           field_mapping={
#                                               "schoolName": "text",
#                                               "degree": "listbox",
#                                               "fieldOfStudy": "multiselect",
#                                               "gradeAverage": "text",
#                                               "startDate-firstYearAttended-dateSectionDay": "date",
#                                               "startDate-firstYearAttended-dateSectionMonth": "date",
#                                               "startDate-firstYearAttended-dateSectionYear": "date",
#                                               "endDate-lastYearAttended-dateSectionDay": "date",
#                                               "endDate-lastYearAttended-dateSectionMonth": "date",
#                                               "endDate-lastYearAttended-dateSectionYear": "date"
#                                           })

#     fill_experience_or_education_sections(driver, wait,
#                                           data_list=form_data_page_2.get("certifications", []),
#                                           section_name="Certifications-",
#                                           field_mapping={
#                                               "certification": "multiselect",
#                                               "certificationNumber": "text",
#                                               "issuedDate-dateSectionDay": "date",
#                                               "issuedDate-dateSectionMonth": "date",
#                                               "issuedDate-dateSectionYear": "date",
#                                               "expirationDate-dateSectionDay": "date",
#                                               "expirationDate-dateSectionMonth": "date",
#                                               "expirationDate-dateSectionYear": "date",
#                                               "attachments": "file"
#                                           })


# # === MAIN FLOW ===

# urls = final_df[final_df["Application Link"].str.contains("myworkdayjobs.com", na=False)]["Application Link"]

# for i, url in enumerate(urls):
#     print(f"\n🌐 Visiting: {url}")
#     send_workday_url(driver, url)
    
#     if i==4 or is_job_page_missing(driver, short_wait):
#         continue

#     if not click_apply_buttons(driver, wait, "//a[contains(text(), 'Apply')]"):
#         continue
#     time.sleep(2)

#     if not click_apply_buttons(driver, wait, "//a[contains(text(), 'Apply Manually')]"):
#         continue
#     time.sleep(2)

#     if not signup_or_signin(driver, wait, profile):
#         print("❌ Failed to handle authentication flow — skipping.")
#         continue
    
#     my_information_page(driver, form_data)
    

#     if not click_apply_buttons(driver, wait, "//button[@data-automation-id='pageFooterNextButton'][contains(text(), 'Save and Continue')]"):
#         continue
    
#     my_experience_form(driver, form_data_page_2)


#     print("✔ Saved form 1.")

#     if pyautogui.confirm(text='Correct??',
#                          title='Manual Auth Required',buttons=['Yes', 'No']) == 'Yes':
#         print("✔ User confirmed authentication.")
#         continue


In [ ]:
# import pandas as pd

# pd.set_option('display.max_colwidth', None)
# print(final_df[final_df["Application Link"].str.contains("myworkdayjobs.com", na=False)]["Application Link"])

In [ ]:
# -------------------{Function to open URL in Workday and clear local/session storage}---------------------------
def send_workday_url(driver, url, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        driver.get(url)
        time.sleep(2)
        print("🌐 Navigated to Workday URL.")
        try:
            driver.delete_all_cookies()
            driver.execute_script("window.localStorage.clear();")
            driver.execute_script("window.sessionStorage.clear();")
            driver.refresh()
            time.sleep(1)
            print("✔ Cleared cookies and local/session storage.")
        except Exception:
            print("⚠️ Failed to clear local/session storage.")
    except Exception:
        print("⚠️ Failed in send_workday_url. Skipping to next.")

# -------------------{Function to detect missing job page error}---------------------------
def is_job_page_missing(driver, short_wait, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        short_wait.until(lambda d: d.find_elements(By.XPATH, "//span[@data-automation-id='errorMessage']"))
        try:
            elements = driver.find_elements(By.XPATH, "//span[@data-automation-id='errorMessage']")
            for el in elements:
                if "doesn't exist" in el.text or "not available" in el.text.lower():
                    print(f"❌ Job error found: {el.text.strip()} — skipping.")
                    return True
        except Exception:
            print("⚠️ Failed while checking error message elements.")
    except TimeoutException:
        print("ℹ️ No missing job error detected.")
    except Exception:
        print("⚠️ Error during is_job_page_missing check.")
    return False

# -------------------{Function to click apply button}---------------------------
def click_apply_buttons(driver, wait, xpath, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        print(f"➡️ Trying to click: {xpath}")
        try:
            button = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", button)
                print("✔ Click successful.")
                return True
            except Exception:
                print("⚠️ Failed to scroll or click the button.")
        except (TimeoutException, NoSuchElementException):
            print(f"⚠️ Timeout or NoSuchElementException: Button not found for xpath: {xpath}")
    except Exception:
        print(f"⚠️ Unexpected error clicking button for xpath: {xpath}")
    return False

# -------------------{Function to tick consent checkbox if present}---------------------------
def tick_checkbox_if_present(driver, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        checkbox = driver.find_element(By.CSS_SELECTOR, "input[data-automation-id='createAccountCheckbox']")
        if not checkbox.is_selected():
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", checkbox)
                print("✔ Ticked consent checkbox.")
                return True
            except Exception:
                print("⚠️ Failed to scroll or click checkbox.")
    except NoSuchElementException:
        print("ℹ️ Consent checkbox not found — skipping.")
    except Exception:
        print("⚠️ Unexpected error ticking consent checkbox.")
    return False

# -------------------{Function to check if page is Sign-In}---------------------------
def is_signin_page(driver):
    try:
        return bool(driver.find_elements(By.XPATH, "//h2[contains(text(), 'Sign In')]"))
    except Exception:
        print("⚠️ Failed checking for Sign-In page.")
        return False

# -------------------{Function to check if page is Create Account}---------------------------
def is_create_account_page(driver):
    try:
        return bool(driver.find_elements(By.XPATH, "//h2[contains(text(), 'Create Account')]"))
    except Exception:
        print("⚠️ Failed checking for Create Account page.")
        return False

# -------------------{Function to click Sign-In switch button}---------------------------
def try_click_signin_switch(driver, wait, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        switch_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-automation-id='signInLink']")))
        driver.execute_script("arguments[0].click();", switch_button)
        print("🔁 Switched to Sign-In form.")
        time.sleep(2)
    except TimeoutException:
        print("⚠️ Switch to Sign-In button not found.")
    except Exception:
        print("⚠️ Unexpected error trying to switch to Sign-In.")

# -------------------{Function to click Create Account switch button}---------------------------
def try_click_create_account_switch(driver, wait, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        switch_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-automation-id='createAccountLink']")))
        driver.execute_script("arguments[0].click();", switch_button)
        print("🔁 Switched to Create Account form.")
        time.sleep(2)
    except TimeoutException:
        print("⚠️ Switch to Create Account button not found.")
    except Exception:
        print("⚠️ Unexpected error trying to switch to Create Account.")

# -------------------{Function to fill email and password fields}---------------------------
def send_email_password(driver, wait, profile, verify_password=False, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        email_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='email'], input[data-automation-id='email']")))
        email_input.send_keys(Keys.CONTROL + "a", Keys.DELETE)
        email_input.send_keys(profile['email'])
        print("📧 Filled email.")
    except TimeoutException:
        print("⚠️ Timeout filling email field.")
    except Exception:
        print("⚠️ Unexpected error filling email field.")

    try:
        password_input = driver.find_element(By.CSS_SELECTOR, "input[type='password'][data-automation-id='password'], input[type='password'][name='password']")
        password_input.send_keys(Keys.CONTROL + "a", Keys.DELETE)
        password_input.send_keys(profile['password'])
        print("🔒 Filled password.")
    except NoSuchElementException:
        print("⚠️ Password field not found.")
    except Exception:
        print("⚠️ Unexpected error filling password field.")

    if verify_password:
        try:
            confirm_password_input = driver.find_element(By.CSS_SELECTOR, "input[type='password'][data-automation-id='verifyPassword'], input[type='password'][name='verifyPassword']")
            confirm_password_input.send_keys(Keys.CONTROL + "a", Keys.DELETE)
            confirm_password_input.send_keys(profile['password'])
            print("🔒 Confirmed password.")
        except NoSuchElementException:
            print("⚠️ Confirm password field not found.")
        except Exception:
            print("⚠️ Unexpected error filling confirm password field.")

    try:
        tick_checkbox_if_present(driver)
    except Exception:
        print("⚠️ Error trying to tick consent checkbox.")

# -------------------{Function to play beep sound as an alert}---------------------------
def alert_user_beep(sound_file='D:/Course/ChatBOT/components/linkedin_automation/jobs_applier_selanium/notification/alert.mp3', repeat=3, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        pygame.mixer.init()
        print("⚠️ Alerting user that it requires manual authentication")
        for _ in range(repeat):
            pygame.mixer.music.load(sound_file)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)
    except Exception:
        print("⚠️ Failed to play alert sound.")

# -------------------{Function to handle manual authentication flow}---------------------------
def handle_manual_authentication(log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        alert_user_beep()
        print("⌛ Waiting for user manual authentication confirmation...")
        start_time = time.time()
        while time.time() - start_time < 300:
            try:
                response = pyautogui.confirm(
                    text='Have you completed manual authentication?',
                    title='Manual Auth Required',
                    buttons=['Yes', 'No']
                )
                if response == 'Yes':
                    print("✔ User confirmed authentication.")
                    return True
                elif response == 'No':
                    print("⏭️ User declined. Skipping job.")
                    return False
                else:
                    time.sleep(5)
            except Exception:
                print("⚠️ Error during manual authentication prompt.")
        print("⏳ Timeout: No manual authentication confirmed.")
    except Exception:
        print("⚠️ Error during manual authentication flow.")
    return False

# -------------------{Function to handle sign-up or sign-in flow}---------------------------
def signup_or_signin(driver, wait, profile, timeout_seconds=60, log_base="logs/job_application_logs/logs_text/", echo=False):
    print("🚀 Attempting to sign up or sign in...")
    start_time = time.time()
    try:
        while time.time() - start_time < timeout_seconds:
            try:
                driver.switch_to.default_content()
            except Exception:
                print("⚠️ Failed to switch to default content.")

            try:
                if is_create_account_page(driver):
                    send_email_password(driver, wait, profile, verify_password=True)
                    try:
                        create_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Create Account']")))
                        driver.execute_script("arguments[0].click();", create_btn)
                        print("📝 Attempted account creation.")
                    except TimeoutException:
                        print("⚠️ Could not click 'Create Account' button.")
                    except Exception:
                        print("⚠️ Unexpected error clicking 'Create Account' button.")
                    time.sleep(2)

                    if is_signin_page(driver):
                        if handle_manual_authentication():
                            send_email_password(driver, wait, profile, verify_password=False)
                            try:
                                signin_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Sign In']")))
                                driver.execute_script("arguments[0].click();", signin_btn)
                                print("🔐 Attempted sign in.")
                            except TimeoutException:
                                print("⚠️ Could not click 'Sign In' button.")
                            except Exception:
                                print("⚠️ Unexpected error clicking 'Sign In' button.")
                            time.sleep(2)
                        else:
                            break

                elif is_signin_page(driver):
                    send_email_password(driver, wait, profile, verify_password=False)
                    try:
                        signin_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@aria-label='Sign In']")))
                        driver.execute_script("arguments[0].click();", signin_btn)
                        print("🔐 Attempted sign in.")
                    except TimeoutException:
                        print("⚠️ Could not click 'Sign In' button.")
                    except Exception:
                        print("⚠️ Unexpected error clicking 'Sign In' button.")
                    time.sleep(2)

                try:
                    if (
                        driver.find_elements(By.XPATH, "//h1[contains(text(), 'Something went wrong')]") or
                        driver.find_elements(By.XPATH, "//h2[contains(text(), 'My Information')]") or
                        driver.find_elements(By.CSS_SELECTOR, "button[data-automation-id='pageFooterNextButton']")
                    ):
                        print("✔ Success: Detected entry into application flow or profile page.")
                        return True
                except Exception:
                    print("⚠️ Error checking page state for success.")

                try:
                    if is_create_account_page(driver):
                        try_click_signin_switch(driver, wait)
                    elif is_signin_page(driver):
                        try_click_create_account_switch(driver, wait)
                except Exception:
                    print("⚠️ Error switching between Create Account and Sign In forms.")

                time.sleep(1)
            except Exception:
                print("⚠️ Unexpected error during signup/signin loop.")

        print("⏳ Timeout: did not reach 'My Information' page.")
    except Exception:
        print("❌ Error during signup/signin process.")
    return False

# -------------------- Helper to delete files ---------------------------------
def delete_existing_files(driver, elem_name, log_base="logs/job_application_logs/logs_text/", echo=False):
    #---------------------{Find all delete buttons using the specified XPath}----------------------
    try: delete_buttons = driver.find_elements(By.XPATH, f"//div[contains(@data-fkit-id, '-{elem_name}')]//button[@type='button' and @data-automation-id='delete-file']")
    #---------------------{If finding delete buttons fails, exit the function}----------------------
    except: return

    #---------------------{Check if no delete buttons were found}----------------------
    if not delete_buttons:
        print(f"⚠️ No files found to delete in '{elem_name}'")
        return

    #---------------------{Iterate over all delete buttons found}----------------------
    for btn in delete_buttons:
        try:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            driver.execute_script("arguments[0].click();", btn)
            WebDriverWait(driver, 2).until(EC.staleness_of(btn))
            print(f"✔ Deleted a file in '{elem_name}'")
        except TimeoutException: print(f"⚠️ Timeout while trying to delete file in '{elem_name}'")
        except Exception: print(f"⚠️ Could not delete file in '{elem_name}'")


# -------------------- Helper to delete selected pills in multiselect ---------------------------------
def delete_selected_pills(driver, elem_name, log_base="logs/job_application_logs/logs_text/", echo=False):
    #---------------------{Try to find all delete icons within the container for pills}----------------------
    try: delete_icons = driver.find_elements(By.XPATH, f"//div[contains(@data-automation-id, '-{elem_name}')]//div[@data-automation-id='DELETE_charm']")
    #---------------------{If the search for delete icons fails, exit the function}----------------------
    except: return

    #---------------------{Check if no selected pills were found}----------------------
    if not delete_icons:
        print(f"⚠️ No selected pills found in '{elem_name}'")
        return

    #---------------------{Iterate over each delete icon found}----------------------
    for icon in delete_icons:
        try:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", icon)
            #---------------------{Click the delete icon to remove the selected pill}----------------------
            driver.execute_script("arguments[0].click();", icon)
            print(f"✔ Cleared a selected value in '{elem_name}'")
        except Exception: print(f"⚠️ Could not delete pill in '{elem_name}'")


# -------------------- select multivalues ---------------------------------
def click_multiselect(driver, wait, element, elem_name, value):    
    try:
        #---------------------{Find the virtualized container for dropdown options}----------------------
        container = driver.find_element(By.CSS_SELECTOR, "div.ReactVirtualized__Grid")

        #---------------------{Build an XPath expression to find the dropdown option with the given value}----------------------
        dropdown_option_xpath = (
            f"//li[@role='option']//div[normalize-space()='{value}']"
            f" | //div[@role='option']//div[@data-automation-id='promptOption' and @data-automation-label='{value}']"
        )
        
        last_scroll_top = -1

        #---------------------{Loop to scroll and search until option is found or scrolling stops}----------------------
        while True:
            try:
                #---------------------{Try to find the dropdown option in the container}----------------------
                dropdown_option = container.find_element(By.XPATH, dropdown_option_xpath)
                
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", dropdown_option)
                time.sleep(1)

                #---------------------{Check the selection state of the option}----------------------
                is_selected = dropdown_option.get_attribute("aria-selected") or dropdown_option.get_attribute("data-automation-selected")

                #---------------------{If selected, print success and exit loop}----------------------
                if is_selected.lower() == "true":                    
                    print(f"✔ Selected multiselect option '{value}' for element '{elem_name}'.")
                    break

                #---------------------{Otherwise, click the option to select it}----------------------
                driver.execute_script("arguments[0].click();", dropdown_option)
                time.sleep(1)
            #---------------------{If dropdown option is not found at this scroll position, continue scrolling}----------------------
            except Exception: pass

            #---------------------{Keep scrolling downwards to reveal more options}----------------------
            current_scroll_top = driver.execute_script("return arguments[0].scrollTop;", container)
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 50;", container)
            time.sleep(0.2)
            new_scroll_top = driver.execute_script("return arguments[0].scrollTop;", container)

            #---------------------{If no new scroll position was reached, exit loop}----------------------
            if new_scroll_top == current_scroll_top or new_scroll_top == last_scroll_top:
                print(f"⚠️ Could not select option '{value}' for element '{elem_name}' after scrolling.")
                break

            last_scroll_top = new_scroll_top

    except Exception: print(f"⚠️ Could not select option '{value}' for element '{elem_name}'.")


# -------------------- Helper Function interact_with ---------------------------------
def interact_with(driver, wait, by, elem_name, value, field_key, elem_type, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        element = None

        #---------------------{Handle text elements}----------------------
        if elem_type == "text":
            try: element = wait.until(EC.visibility_of_element_located((by, elem_name)))
            except TimeoutException: print(f"⚠️ Timeout: Text element '{elem_name}' not found.")
        #---------------------{Handle checkbox, listbox, multiselect elements}----------------------
        elif elem_type in ["checkbox", "listbox", "multiselect"]:
            try: element = wait.until(EC.presence_of_element_located((by, elem_name)))
            except TimeoutException: print(f"⚠️ Timeout: Element '{elem_name}' not found.")
        #---------------------{Handle file elements}----------------------
        elif elem_type == "file":
            try: element = wait.until(EC.presence_of_element_located((by, elem_name)))
            except TimeoutException: print(f"⚠️ Timeout: File input '{elem_name}' not found.")
        #---------------------{Handle date elements}----------------------
        elif elem_type == "date":
            try:
                display_xpath = f"{elem_name}//div[contains(@id,'{field_key}-display')]"
                input_xpath = f"{elem_name}//input[contains(@id,'{field_key}-input')]"
                try:
                    #---------------------{Wait until display element is clickable and click it}----------------------
                    display_element = wait.until(EC.element_to_be_clickable((By.XPATH, display_xpath)))
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", display_element)
                    driver.execute_script("arguments[0].click();", display_element)
                except TimeoutException: print(f"⚠️ Timeout: Display element '{elem_name}' not found or not clickable.")
                
                #---------------------{Wait until input element is visible}----------------------
                element = wait.until(EC.visibility_of_element_located((By.XPATH, input_xpath)))
            except TimeoutException: print(f"⚠️ Timeout: Date input '{elem_name}' not found.")
        #---------------------{Otherwise break the operation}----------------------
        else:
            print(f"⚠️ Unknown element type '{elem_type}' for {elem_name}. Skipped.")
            return None

        #---------------------{Scroll element into view}----------------------
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)

        #---------------------{Handle different element type actions}----------------------
        #---------------------{Handle checkbox actions}----------------------
        if elem_type == "checkbox":
            return element        
        #---------------------{Handle file actions}----------------------
        elif elem_type == "file":
            #---------------------{Delete any existing files in field}----------------------
            delete_existing_files(driver, field_key, log_base, echo)
            try:
                #---------------------{Make the file input field visible and upload the file}----------------------
                driver.execute_script("""
                    arguments[0].style.display = 'block';
                    arguments[0].style.visibility = 'visible';
                    arguments[0].style.opacity = '1';
                    arguments[0].style.position = 'static';
                """, element)
                element.send_keys(value)
                time.sleep(0.5)
                print(f"✔ Uploaded file: {value}")
            except Exception: print(f"⚠️ Could not upload file: {value}")
        #---------------------{Handle multiselect actions}----------------------
        elif elem_type == "multiselect":
            #---------------------{Delete any existing selected pills in multiselect}----------------------
            delete_selected_pills(driver, field_key, log_base, echo)
            try:
                if isinstance(value, list):
                    #---------------------{Loop through each value in the list}----------------------
                    for val in value:
                        driver.execute_script("arguments[0].click();", element)
                        #---------------------{Reset the input value using JS}----------------------
                        driver.execute_script("arguments[0].value = '';", element)
                        time.sleep(1)  # Let React register

                        #---------------------{Send the value and press ENTER to search}----------------------
                        element.send_keys(val)
                        element.send_keys(Keys.ENTER)
                        time.sleep(1)

                        #---------------------{Manually update value via JS}----------------------
                        driver.execute_script(f"arguments[0].value = '{val}';", element)
                        time.sleep(1)
                        element.send_keys(Keys.TAB)
                else:
                    #---------------------{Single value case for multiselect}----------------------
                    element.send_keys(value)
                    time.sleep(0.1)
                    element.send_keys(Keys.ENTER)
                    click_multiselect(driver, wait, element, field_key, value)
                print(f"✔ Filled {field_key} with '{value}'")
            except Exception: print(f"⚠️ Could not interact with multiselect: {value}")
        #---------------------{Handle text, listbox, date actions}----------------------
        elif elem_type in ["text", "listbox", "date"]:
            try:
                if elem_type == "date": 
                    #---------------------{Clear existing value in date input}----------------------
                    element.clear()
                else:
                    #---------------------{Clear text input field}----------------------
                    driver.execute_script("arguments[0].click();", element)
                    element.send_keys(Keys.CONTROL + 'a')
                    element.send_keys(Keys.DELETE)
                time.sleep(1)
                #---------------------{Send the new value to the element}----------------------
                element.send_keys(value)

                #---------------------{Handle pressing ENTER or TAB as needed}----------------------
                if elem_type == "listbox": element.send_keys(Keys.ENTER)
                elif elem_type != "date": element.send_keys(Keys.TAB)

                print(f"✔ Filled {elem_name} with '{value}'")
            except Exception: print(f"⚠️ Could not fill element '{elem_name}' with value '{value}'.")
    except Exception:
        print(f"⚠️ Could not process interact_with for element '{elem_name}'.")


# -------------------- fill_experience_or_education_sections ---------------------------------
def fill_experience_or_education_sections(driver, wait, short_wait, data_list, section_name, field_mapping, heading="My Experience", log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        #---------------------{Check if heading includes skill or resume}----------------------
        check_headings = "skill" not in heading.lower() and "resume" not in heading.lower()

        #---------------------{Locate the heading element}----------------------
        xpath_base = f"//h3[contains(text(), '{heading}')]"
        if not click_apply_buttons(driver, wait, xpath_base):
            print(f"Unable to find {heading}. Continuing with the next part")
            return None

        #---------------------{If no skill or resume in headings, try clicking Add button}----------------------
        if check_headings:
            try:
                add_button_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}section']//button[@data-automation-id='add-button' and (text()='Add')]"
                click_apply_buttons(driver, wait, add_button_xpath)
                print(f"✔ Clicked 'Add' for {section_name}.")
            except Exception:
                print(f"⚠️ Failed to click 'Add' button for {section_name}.")

        #---------------------{Iterate through the data list to fill each section}----------------------
        for index, entry in enumerate(data_list, start=1):
            try:
                #---------------------{Check if section exists or needs adding}----------------------
                if check_headings:
                    section_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}{index}-panel']"
                    short_wait.until(lambda d: True)
                    try:
                        h4_elements = driver.find_elements(By.XPATH, f"//h4[@id='{section_name}{index}-panel']")
                        if h4_elements and h4_elements[0].is_displayed():
                            print(f"✔ {section_name}{index} section already exists. Skipping adding.")
                        else:
                            add_another_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}section']//button[contains(@data-automation-id, 'add-button') and (text()='Add Another')]"
                            if click_apply_buttons(driver, wait, add_another_xpath):
                                print(f"✔ Clicked 'Add Another' for {section_name}{index}.")
                    except Exception:
                        print(f"⚠️ Error clicking 'Add Another' for {section_name}{index}.")
                else: 
                    section_xpath = f"//div[@role='group' and @aria-labelledby='{section_name}section']"

                #---------------------{Iterate through each field key in field mapping}----------------------
                for field_key, field_type in field_mapping.items():
                    try:
                        #---------------------{Handle date field separately}----------------------
                        if field_type == "date":
                            date_field_name = field_key.split('-')[0]
                            date_value = entry.get(date_field_name, "")
                            if not date_value: continue
                            day, month, year = date_value.split("/")
                            value = {"dateSectionDay": day, "dateSectionMonth": month, "dateSectionYear": year}.get(field_key.split('-')[-1], "")
                            if not value: continue

                            interact_with(driver=driver, wait=short_wait, by=By.XPATH, elem_name=section_xpath, value=value, field_key=field_key, elem_type="date", log_base=log_base, echo=echo)
                        #---------------------{Handle file field type}----------------------
                        elif field_type == "file":
                            element_xpath = f"{section_xpath}//input[@data-automation-id='file-upload-input-ref' and @type='file']"
                            interact_with(driver=driver, wait=wait, by=By.XPATH, elem_name=element_xpath, value=entry[field_key], field_key=field_key, elem_type="file", log_base=log_base, echo=echo)
                        #---------------------{Handle listbox field type}----------------------
                        elif field_type == "listbox":
                            element_xpath = f"{section_xpath}//button[contains(@id,'--{field_key}') and contains(@name,'{field_key}')]"
                            interact_with(driver=driver, wait=wait, by=By.XPATH, elem_name=element_xpath, value=entry[field_key], field_key=field_key, elem_type="listbox", log_base=log_base, echo=echo)
                        #---------------------{Handle multiselect field type}----------------------
                        elif field_type == "multiselect":
                            element_xpath = f"{section_xpath}//input[contains(@id,'--{field_key}')]"
                            interact_with(driver=driver, wait=wait, by=By.XPATH, elem_name=element_xpath, value=entry[field_key], field_key=field_key, elem_type="multiselect", log_base=log_base, echo=echo)
                        #---------------------{Handle checkbox field type and click if needed}----------------------
                        elif field_type == "checkbox":
                            element_xpath = f"{section_xpath}//input[@type='checkbox' and contains(@id,'--{field_key}') and contains(@name,'{field_key}')]"
                            checkbox = interact_with(driver=driver, wait=short_wait, by=By.XPATH, elem_name=element_xpath, value=entry[field_key], field_key=field_key, elem_type="checkbox", log_base=log_base, echo=echo)
                            if entry[field_key] != (checkbox.get_attribute("aria-checked") == "true"):
                                driver.execute_script("arguments[0].click();", checkbox)
                                print(f"✔ Set '{field_key}' checkbox for {section_name}{index}.")
                        #---------------------{Handle other text field types}----------------------
                        else:
                            element_xpath = f"{section_xpath}//{'textarea' if field_key == 'roleDescription' else 'input'}[contains(@id,'--{field_key}')]"
                            interact_with(driver=driver, wait=wait, by=By.XPATH, elem_name=element_xpath, value=entry[field_key], field_key=field_key, elem_type=field_type, log_base=log_base, echo=echo)
                        print(f"✔ Finished {section_name}{index}.")
                    except Exception:
                        print(f"⚠️ Could not fill field {field_key} for {section_name}{index}.")
            except Exception:
                print(f"⚠️ Error filling {section_name}{index}.")
    except Exception:
        print("⚠️ Unexpected error in fill_experience_or_education_sections.")


# -------------------- my_experience_form ---------------------------------
def my_experience_form(driver, wait, short_wait, my_experience_data, retry_count=0, max_retries=1, log_base="logs/job_application_logs/logs_text/", echo=False):
    try:
        #---------------------{Start the page 2 filling process}----------------------
        print("🌟 Starting Page 2 filling process...")
        time.sleep(5)

        #---------------------{Locate the 'My Experience' tab heading}----------------------
        xpath_base = f"//h2[contains(text(), 'My Experience')]"
        if not click_apply_buttons(driver=driver, wait=wait, xpath=xpath_base, log_base=log_base, echo=echo):
            print("❌ Unable to find my experience tab.")
            return None

        #---------------------{Fill the Work Experience section}----------------------
        fill_experience_or_education_sections(driver=driver, wait=wait, short_wait=short_wait, data_list=my_experience_data.get("workExperience", []), section_name="Work-Experience-", field_mapping=workexperience_field_mapping, heading="Work Experience", log_base=log_base, echo=echo)
        
        #---------------------{Fill the Education section}----------------------
        fill_experience_or_education_sections(driver=driver, wait=wait, short_wait=short_wait, data_list=my_experience_data.get("education", []), section_name="Education-", field_mapping=education_field_mapping, heading="Education", log_base=log_base, echo=echo)
        
        #---------------------{Fill the Certifications section}----------------------
        fill_experience_or_education_sections(driver=driver, wait=wait, short_wait=short_wait, data_list=my_experience_data.get("certifications", []), section_name="Certifications-", field_mapping=certification_field_mapping, heading="Certifications", log_base=log_base, echo=echo)
        
        #---------------------{Fill the Languages section}----------------------
        fill_experience_or_education_sections(driver=driver, wait=wait, short_wait=short_wait, data_list=my_experience_data.get("languages", []), section_name="Languages-", field_mapping=language_field_mapping, heading="Languages", log_base=log_base, echo=echo)
        
        #---------------------{Fill the Skills section}----------------------
        # fill_experience_or_education_sections(driver=driver, wait=wait, short_wait=short_wait, data_list=my_experience_data.get("skills-section", section_name=[]), "Skills-", field_mapping=skill_field_mapping, heading="Skills", log_base=log_base, echo=echo)

        #---------------------{Fill the Resume/CV section}----------------------
        fill_experience_or_education_sections(driver=driver, wait=wait, short_wait=short_wait, data_list=my_experience_data.get("resume", []), section_name="Resume/CV-", field_mapping=resume_field_mapping, heading="Resume/CV", log_base=log_base, echo=echo)

    except Exception:
        #---------------------{Handle errors during execution}----------------------
        print("⚠️ Error occurred during my_experience_form execution.")
        
        #---------------------{Check retry count and retry if allowed}----------------------
        if retry_count < max_retries:
            print(f"🔄 Retrying My Experience Form (attempt {retry_count + 1}/{max_retries})...")
            time.sleep(2)
            try:
                #---------------------{Navigate back to retry the form}----------------------
                if click_apply_buttons(driver=driver, wait=wait, xpath="//button[@data-automation-id='pageFooterNextButton' and contains(text(), 'Back')]", log_base=log_base, echo=echo):
                    print("🔄 Clicked Back button.")
                time.sleep(2)
                
                #---------------------{Try to click 'Save and Continue' to refresh the page}----------------------
                if click_apply_buttons(driver=driver, wait=wait, xpath="//button[@data-automation-id='pageFooterNextButton' and contains(text(), 'Save and Continue')]", log_base=log_base, echo=echo):
                    print("🔄 Clicked Save and Continue button.")
                time.sleep(2)
            except Exception:
                print(f"⚠️ Retry navigation error")
            
            #---------------------{Recursive call with incremented retry count}----------------------
            my_experience_form(driver=driver, wait=wait, short_wait=short_wait, my_experience_data=my_experience_data, log_base=log_base, echo=echo, retry_count=retry_count + 1, max_retries=max_retries)
        else:
            print("❌ Maximum retries reached for My Experience Form.")

# urls = final_df[final_df["Application Link"].str.contains("myworkdayjobs.com", na=False)]["Application Link"]

# send_workday_url(driver, urls[31])

# is_job_page_missing(driver, short_wait)

# click_apply_buttons(driver, wait, "//a[contains(text(), 'Apply')]")
# time.sleep(2)

# click_apply_buttons(driver, wait, "//a[contains(text(), 'Apply Manually')]")
# time.sleep(2)

# signup_or_signin(driver, wait, profile)

# # my_information_page(driver, wait, form_data=my_information_data)
# click_apply_buttons(driver, wait, "//button[@data-automation-id='pageFooterNextButton']", 
                            #    "Save and Continue")
my_experience_form(driver, wait, short_wait, my_experience_data)

🌟 Starting Page 2 filling process...
➡️ Trying to click: //h2[contains(text(), 'My Experience')]
✔ Click successful.
➡️ Trying to click: //h3[contains(text(), 'Work Experience')]
✔ Click successful.
➡️ Trying to click: //div[@role='group' and @aria-labelledby='Work-Experience-section']//button[@data-automation-id='add-button' and (text()='Add')]
⚠️ Timeout or NoSuchElementException: Button not found for xpath: //div[@role='group' and @aria-labelledby='Work-Experience-section']//button[@data-automation-id='add-button' and (text()='Add')]
✔ Clicked 'Add' for Work-Experience-.
✔ Work-Experience-1 section already exists. Skipping adding.
✔ Filled //div[@role='group' and @aria-labelledby='Work-Experience-1-panel']//input[contains(@id,'--jobTitle')] with 'Software Engineer'
✔ Finished Work-Experience-1.
✔ Filled //div[@role='group' and @aria-labelledby='Work-Experience-1-panel']//input[contains(@id,'--companyName')] with 'ABC Corp'
✔ Finished Work-Experience-1.
✔ Filled //div[@role='group' a

In [ ]:
def fetch_all_certifications_and_suboptions(driver, wait, multiselect_input_xpath):
    """
    Fetches all main certifications and their nested sub-certifications from a Workday multi-select dropdown.
    Handles dynamic rendering for both main options and sub-options by collecting visible items at each scroll step.
    """
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
    import time

    dropdown_data = {}

    try:
        # Click the multiselect input to open the dropdown
        element = wait.until(EC.presence_of_element_located((By.XPATH, multiselect_input_xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
        time.sleep(1)

        # Fetch all main labels
        main_labels = set()
        last_main_count = -1
        scroll_index = 0
        scroll_step = 32  # approx row height

        while True:
            scroll_container = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//div[contains(@class,'ReactVirtualized__Grid__innerScrollContainer')]")))
            driver.execute_script("arguments[0].scrollTop = arguments[1];", scroll_container, scroll_index * scroll_step)
            time.sleep(1)

            main_option_elements = driver.find_elements(
                By.XPATH,
                "//div[@data-automation-id='promptOption' and ancestor::div[@data-uxi-multiselectlistitem-index]]"
            )

            for item in main_option_elements:
                try:
                    label = item.get_attribute("data-automation-label").strip()
                    if label:
                        main_labels.add(label)
                except StaleElementReferenceException:
                    continue

            if len(main_labels) == last_main_count:
                break
            last_main_count = len(main_labels)
            scroll_index += 1

        print(f"✔ Found {len(main_labels)} main options.")

        # Now loop through each main option by label (not index)
        for label in main_labels:
            try:
                item_xpath = f"//div[@data-automation-id='promptOption' and @data-automation-label='{label}']"
                item_element = wait.until(EC.presence_of_element_located((By.XPATH, item_xpath)))

                driver.execute_script("arguments[0].scrollIntoView(true);", item_element)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", item_element)
                time.sleep(1)

                suboption_labels = set()
                last_sub_count = -1
                sub_scroll_index = 0

                while True:
                    sub_scroll_container = wait.until(EC.presence_of_element_located(
                        (By.XPATH, "//div[contains(@class,'ReactVirtualized__Grid__innerScrollContainer')]")))
                    driver.execute_script("arguments[0].scrollTop = arguments[1];", sub_scroll_container, sub_scroll_index * scroll_step)
                    time.sleep(1)

                    suboption_elements = driver.find_elements(
                        By.XPATH,
                        "//div[@data-automation-id='promptLeafNode']//div[@data-automation-id='promptOption']"
                    )

                    for sub in suboption_elements:
                        try:
                            sub_label = sub.get_attribute("data-automation-label").strip()
                            if sub_label and sub_label != label:
                                suboption_labels.add(sub_label)
                        except StaleElementReferenceException:
                            continue

                    if len(suboption_labels) == last_sub_count:
                        break
                    last_sub_count = len(suboption_labels)
                    sub_scroll_index += 1

                print(f"✔ Processed '{label}': {len(suboption_labels)} sub-options found.")
                dropdown_data[label] = list(suboption_labels)

                # Click back button
                try:
                    back_button_xpath = "//div[@data-automation-id='multiSelectHeader']//span[@data-automation-id='backButton']"
                    back_btn = driver.find_element(By.XPATH, back_button_xpath)
                    driver.execute_script("arguments[0].click();", back_btn)
                    time.sleep(1)
                except NoSuchElementException:
                    pass

            except TimeoutException:
                print(f"⚠️ Skipping '{label}' (element not found).")
                continue
            except Exception as e:
                print(f"⚠️ Error processing '{label}': {e}")
                continue

        print("✔ Completed all items.")
        return dropdown_data

    except TimeoutException:
        print("⚠️ Timeout: Could not locate multiselect input.")
        return {}
    except Exception as e:
        print(f"⚠️ Error fetching dropdown options: {e}")
        return {}

# Usage Example:
multiselect_xpath = "//input[contains(@id,'--degree')]"
dropdown_options = fetch_all_certifications_and_suboptions(driver, wait, multiselect_xpath)

print("Dropdown Options with Sub-Options:", dropdown_options)